In [1]:
import os
import re
import time
import torch
import torch
import random
import bisect
import json
from pathlib import Path
from tokenizers import Tokenizer
from transformers import GPT2Model, GPT2LMHeadModel, GPT2Config, LlamaModel, LlamaForCausalLM, PreTrainedModel 
from samplings import top_p_sampling, top_k_sampling, temperature_sampling
from abctoolkit.utils import Exclaim_re, Quote_re, SquareBracket_re, Barline_regexPattern
from abctoolkit.transpose import Note_list, Pitch_sign_list
from abctoolkit.duration import calculate_bartext_duration

In [19]:
# Configurations for inference
INFERENCE_WEIGHTS_PATH = '../weights/weights_notagenx_p_size_16_p_length_1024_p_layers_20_h_size_1280.pth'               # Path to weights for inference# Folder to save output files
TOP_K = 9                                                       # Top k for sampling
TOP_P = 0.9                                                      # Top p for sampling
TEMPERATURE = 1.2                                                 # Temperature for sampling

# Configurations for model
PATCH_STREAM = True                                             # Stream training / inference
PATCH_SIZE = 16                                                # Patch Size
PATCH_LENGTH = 1024                                             # Patch Length
CHAR_NUM_LAYERS = 6                                             # Number of layers in the decoder
PATCH_NUM_LAYERS = 20                                           # Number of layers in the encoder
HIDDEN_SIZE = 1280                                               # Hidden Size

device = torch.device("cuda")

In [20]:
class Patchilizer:
    def __init__(self, stream=PATCH_STREAM):
        self.stream = stream
        self.delimiters = ["|:", "::", ":|", "[|", "||", "|]", "|"]
        self.regexPattern = '(' + '|'.join(map(re.escape, self.delimiters)) + ')'
        self.bos_token_id = 1
        self.eos_token_id = 2
        self.special_token_id = 0

    def split_bars(self, body_lines):
        """
        Split a body of music into individual bars.
        """
        new_bars = []
        try:
            for line in body_lines:
                line_bars = re.split(self.regexPattern, line)
                line_bars = list(filter(None, line_bars))
                new_line_bars = []

                if len(line_bars) == 1:
                    new_line_bars = line_bars
                else:
                    if line_bars[0] in self.delimiters:
                        new_line_bars = [line_bars[i] + line_bars[i + 1] for i in range(0, len(line_bars), 2)]
                    else:
                        new_line_bars = [line_bars[0]] + [line_bars[i] + line_bars[i + 1] for i in range(1, len(line_bars), 2)]
                    if 'V' not in new_line_bars[-1]:
                        new_line_bars[-2] += new_line_bars[-1]  # 吸收最后一个 小节线+\n 的组合
                        new_line_bars = new_line_bars[:-1]
                new_bars += new_line_bars
        except:
            pass

        return new_bars

    def split_patches(self, abc_text, patch_size=PATCH_SIZE, generate_last=False):
        if not generate_last and len(abc_text) % patch_size != 0:
            abc_text += chr(self.eos_token_id)
        patches = [abc_text[i : i + patch_size] for i in range(0, len(abc_text), patch_size)]
        return patches

    def patch2chars(self, patch):
        """
        Convert a patch into a bar.
        """
        bytes = ''
        for idx in patch:
            if idx == self.eos_token_id:
                break
            if idx < self.eos_token_id:
                pass
            bytes += chr(idx)
        return bytes
        

    def patchilize_metadata(self, metadata_lines):

        metadata_patches = []
        for line in metadata_lines:
            metadata_patches += self.split_patches(line)

        return metadata_patches
    
    def patchilize_tunebody(self, tunebody_lines, encode_mode='train'):

        tunebody_patches = []
        bars = self.split_bars(tunebody_lines)
        if encode_mode == 'train':
            for bar in bars:
                tunebody_patches += self.split_patches(bar)
        elif encode_mode == 'generate':
            for bar in bars[:-1]:
                tunebody_patches += self.split_patches(bar)
            tunebody_patches += self.split_patches(bars[-1], generate_last=True)
       
        return tunebody_patches

    def encode_train(self, abc_text, patch_length=PATCH_LENGTH, patch_size=PATCH_SIZE, add_special_patches=True, cut=True):

        lines = abc_text.split('\n')
        lines = list(filter(None, lines))
        lines = [line + '\n' for line in lines]

        tunebody_index = -1
        for i, line in enumerate(lines):
            if '[V:' in line:
                tunebody_index = i
                break

        metadata_lines = lines[ : tunebody_index]
        tunebody_lines = lines[tunebody_index : ]

        if self.stream:
            tunebody_lines = ['[r:' + str(line_index) + '/' + str(len(tunebody_lines) - line_index - 1) + ']' + line for line_index, line in
                                enumerate(tunebody_lines)]    

        metadata_patches = self.patchilize_metadata(metadata_lines)
        tunebody_patches = self.patchilize_tunebody(tunebody_lines, encode_mode='train')

        if add_special_patches:
            bos_patch = chr(self.bos_token_id) * (patch_size - 1) + chr(self.eos_token_id)
            eos_patch = chr(self.bos_token_id) + chr(self.eos_token_id) * (patch_size - 1)

            metadata_patches = [bos_patch] + metadata_patches
            tunebody_patches = tunebody_patches + [eos_patch]

        if self.stream:
            if len(metadata_patches) + len(tunebody_patches) > patch_length:
                available_cut_indexes = [0] + [index + 1 for index, patch in enumerate(tunebody_patches) if '\n' in patch]
                line_index_for_cut_index = list(range(len(available_cut_indexes)))  
                end_index = len(metadata_patches) + len(tunebody_patches) - patch_length
                biggest_index = bisect.bisect_left(available_cut_indexes, end_index) 
                available_cut_indexes = available_cut_indexes[:biggest_index + 1]

                if len(available_cut_indexes) == 1:
                    choices = ['head']
                elif len(available_cut_indexes) == 2:
                    choices = ['head', 'tail']
                else:
                    choices = ['head', 'tail', 'middle']
                choice = random.choice(choices)
                if choice == 'head':
                    patches = metadata_patches + tunebody_patches[0:]
                else:
                    if choice == 'tail':
                        cut_index = len(available_cut_indexes) - 1
                    else:
                        cut_index = random.choice(range(1, len(available_cut_indexes) - 1))

                    line_index = line_index_for_cut_index[cut_index] 
                    stream_tunebody_lines = tunebody_lines[line_index : ]
                    
                    stream_tunebody_patches = self.patchilize_tunebody(stream_tunebody_lines, encode_mode='train')
                    if add_special_patches:
                        stream_tunebody_patches = stream_tunebody_patches + [eos_patch]
                    patches = metadata_patches + stream_tunebody_patches
            else:
                patches = metadata_patches + tunebody_patches
        else:
            patches = metadata_patches + tunebody_patches

        if cut: 
            patches = patches[ : patch_length]
        else:   
            pass

        # encode to ids
        id_patches = []
        for patch in patches:
            id_patch = [ord(c) for c in patch] + [self.special_token_id] * (patch_size - len(patch))
            id_patches.append(id_patch)

        return id_patches

    def encode_generate(self, abc_code, patch_length=PATCH_LENGTH, patch_size=PATCH_SIZE, add_special_patches=True):

        lines = abc_code.split('\n')
        lines = list(filter(None, lines))
    
        tunebody_index = None
        for i, line in enumerate(lines):
            if line.startswith('[V:') or line.startswith('[r:'):
                tunebody_index = i
                break
    
        metadata_lines = lines[ : tunebody_index]
        tunebody_lines = lines[tunebody_index : ]   
    
        metadata_lines = [line + '\n' for line in metadata_lines]
        if self.stream:
            if not abc_code.endswith('\n'):
                tunebody_lines = [tunebody_lines[i] + '\n' for i in range(len(tunebody_lines) - 1)] + [tunebody_lines[-1]]
            else:
                tunebody_lines = [tunebody_lines[i] + '\n' for i in range(len(tunebody_lines))]
        else:
            tunebody_lines = [line + '\n' for line in tunebody_lines]
    
        metadata_patches = self.patchilize_metadata(metadata_lines)
        tunebody_patches = self.patchilize_tunebody(tunebody_lines, encode_mode='generate')
    
        if add_special_patches:
            bos_patch = chr(self.bos_token_id) * (patch_size - 1) + chr(self.eos_token_id)

            metadata_patches = [bos_patch] + metadata_patches
    
        patches = metadata_patches + tunebody_patches
        patches = patches[ : patch_length]

        # encode to ids
        id_patches = []
        for patch in patches:
            if len(patch) < PATCH_SIZE and patch[-1] != chr(self.eos_token_id):
                id_patch = [ord(c) for c in patch]
            else:
                id_patch = [ord(c) for c in patch] + [self.special_token_id] * (patch_size - len(patch))
            id_patches.append(id_patch)
        
        return id_patches

    def decode(self, patches):
        """
        Decode patches into music.
        """
        return ''.join(self.patch2chars(patch) for patch in patches)


class PatchLevelDecoder(PreTrainedModel):
    """
    A Patch-level Decoder model for generating patch features in an auto-regressive manner. 
    It inherits PreTrainedModel from transformers.
    """
    def __init__(self, config):
        super().__init__(config)
        self.patch_embedding = torch.nn.Linear(PATCH_SIZE * 128, config.n_embd)
        torch.nn.init.normal_(self.patch_embedding.weight, std=0.02)
        self.base = GPT2Model(config)

    def forward(self,
                patches: torch.Tensor,
                masks=None) -> torch.Tensor:
        """
        The forward pass of the patch-level decoder model.
        :param patches: the patches to be encoded
        :param masks: the masks for the patches
        :return: the encoded patches
        """
        patches = torch.nn.functional.one_hot(patches, num_classes=128).to(self.dtype)
        patches = patches.reshape(len(patches), -1, PATCH_SIZE * (128))
        patches = self.patch_embedding(patches.to(self.device))

        if masks==None:
            return self.base(inputs_embeds=patches)
        else:
            return self.base(inputs_embeds=patches,
                             attention_mask=masks)


class CharLevelDecoder(PreTrainedModel):
    """
    A Char-level Decoder model for generating the chars within each patch in an auto-regressive manner
    based on the encoded patch features. It inherits PreTrainedModel from transformers.
    """
    def __init__(self, config):
        super().__init__(config)
        self.special_token_id = 0
        self.bos_token_id = 1

        self.base = GPT2LMHeadModel(config)

    def forward(self,
                encoded_patches: torch.Tensor,
                target_patches: torch.Tensor):
        """
        The forward pass of the char-level decoder model.
        :param encoded_patches: the encoded patches
        :param target_patches: the target patches
        :return: the output of the model
        """
        # preparing the labels for model training
        target_patches = torch.cat((torch.ones_like(target_patches[:,0:1])*self.bos_token_id, target_patches), dim=1)
        # print('target_patches shape:', target_patches.shape)

        target_masks = target_patches == self.special_token_id
        labels = target_patches.clone().masked_fill_(target_masks, -100)

        # masking the labels for model training
        target_masks = torch.ones_like(labels)
        target_masks = target_masks.masked_fill_(labels == -100, 0)

        # select patches
        if PATCH_SAMPLING_BATCH_SIZE!=0 and PATCH_SAMPLING_BATCH_SIZE<target_patches.shape[0]:
            indices = list(range(len(target_patches)))
            random.shuffle(indices)
            selected_indices = sorted(indices[:PATCH_SAMPLING_BATCH_SIZE])

            target_patches = target_patches[selected_indices,:]
            target_masks = target_masks[selected_indices,:]
            encoded_patches = encoded_patches[selected_indices,:]

        # get input embeddings
        inputs_embeds = torch.nn.functional.embedding(target_patches, self.base.transformer.wte.weight)

        # concatenate the encoded patches with the input embeddings
        inputs_embeds = torch.cat((encoded_patches.unsqueeze(1), inputs_embeds[:,1:,:]), dim=1)

        output = self.base(inputs_embeds=inputs_embeds, 
                         attention_mask=target_masks,
                         labels=labels)
                         # output_hidden_states=True=True)

        return output

    def generate(self,
                 encoded_patch: torch.Tensor,   # [hidden_size]
                 tokens: torch.Tensor): # [1]
        """
        The generate function for generating a patch based on the encoded patch and already generated tokens.
        :param encoded_patch: the encoded patch
        :param tokens: already generated tokens in the patch
        :return: the probability distribution of next token
        """
        encoded_patch = encoded_patch.reshape(1, 1, -1) # [1, 1, hidden_size]
        tokens = tokens.reshape(1, -1)

        # Get input embeddings
        tokens = torch.nn.functional.embedding(tokens, self.base.transformer.wte.weight)

        # Concatenate the encoded patch with the input embeddings
        tokens = torch.cat((encoded_patch, tokens[:,1:,:]), dim=1)
        
        # Get output from model
        outputs = self.base(inputs_embeds=tokens)
        
        # Get probabilities of next token
        probs = torch.nn.functional.softmax(outputs.logits.squeeze(0)[-1], dim=-1)

        return probs

class NotaGenLMHeadModel(PreTrainedModel):
    """
    NotaGen is a language model with a hierarchical structure.
    It includes a patch-level decoder and a char-level decoder.
    The patch-level decoder is used to generate patch features in an auto-regressive manner.
    The char-level decoder is used to generate the chars within each patch in an auto-regressive manner.
    It inherits PreTrainedModel from transformers.
    """
    def __init__(self, encoder_config, decoder_config):
        super().__init__(encoder_config)
        self.special_token_id = 0
        self.bos_token_id = 1
        self.eos_token_id = 2
        self.patch_level_decoder = PatchLevelDecoder(encoder_config)
        self.char_level_decoder = CharLevelDecoder(decoder_config)

    def forward(self,
                patches: torch.Tensor,
                masks: torch.Tensor):
        """
        The forward pass of the bGPT model.
        :param patches: the patches to be encoded
        :param masks: the masks for the patches
        :return: the decoded patches
        """
        patches = patches.reshape(len(patches), -1, PATCH_SIZE)
        encoded_patches = self.patch_level_decoder(patches, masks)["last_hidden_state"]
        
        left_shift_masks = masks * (masks.flip(1).cumsum(1).flip(1) > 1)
        masks[:, 0] = 0
        
        encoded_patches = encoded_patches[left_shift_masks == 1]
        patches = patches[masks == 1]        

        return self.char_level_decoder(encoded_patches, patches)
        
    def generate(self,
                 patches: torch.Tensor,
                 top_k=0,
                 top_p=1,
                 temperature=1.0):
        """
        The generate function for generating patches based on patches.
        :param patches: the patches to be encoded
        :param top_k: the top k for sampling
        :param top_p: the top p for sampling
        :param temperature: the temperature for sampling
        :return: the generated patches
        """
        if patches.shape[-1] % PATCH_SIZE != 0:
            tokens = patches[:,:,-(patches.shape[-1]%PATCH_SIZE):].squeeze(0, 1)
            tokens = torch.cat((torch.tensor([self.bos_token_id], device=self.device), tokens), dim=-1)
            patches = patches[:,:,:-(patches.shape[-1]%PATCH_SIZE)]
        else:
            tokens =  torch.tensor([self.bos_token_id], device=self.device)

        patches = patches.reshape(len(patches), -1, PATCH_SIZE) # [bs, seq, patch_size]
        encoded_patches = self.patch_level_decoder(patches)["last_hidden_state"]    # [bs, seq, hidden_size]
        generated_patch = []            

        while True:
            prob = self.char_level_decoder.generate(encoded_patches[0][-1], tokens).cpu().detach().numpy()  # [128]
            prob = top_k_sampling(prob, top_k=top_k, return_probs=True) # [128]
            prob = top_p_sampling(prob, top_p=top_p, return_probs=True) # [128]
            token = temperature_sampling(prob, temperature=temperature) # int
            char = chr(token)
            generated_patch.append(token)

            if len(tokens) >= PATCH_SIZE:# or token == self.eos_token_id:
                break
            else:
                tokens = torch.cat((tokens, torch.tensor([token], device=self.device)), dim=0)
        
        return generated_patch

def clean_to_abc(raw_text, unreduce=True, output_path='output.abc'):
    # Remove [r:x/y] tags
    cleaned = re.sub(r'\[r:\d+/\d+\]', '', raw_text)

    # Add required ABC headers
    lines = cleaned.strip().splitlines()
    header_inserted = False
    abc_lines = []
    for line in lines:
        if not header_inserted and line.startswith('%%score'):
            abc_lines.insert(0, 'T:Generated\n')
            abc_lines.insert(0, 'X:1\n')
            header_inserted = True
        abc_lines.append(line if line.endswith('\n') else line + '\n')

    # Optional: fill missing rests
    if unreduce:
        try:
            abc_lines = rest_unreduce(abc_lines)
        except Exception as e:
            print("Unreduce failed:", e)

    # Save to .abc file
    Path(output_path).write_text(''.join(abc_lines), encoding='utf-8')
    print(f"Saved cleaned ABC to {output_path}")
    return output_path

In [21]:
Note_list = Note_list + ['z', 'x']

patchilizer = Patchilizer()

patch_config = GPT2Config(num_hidden_layers=PATCH_NUM_LAYERS,
                          max_length=PATCH_LENGTH,
                          max_position_embeddings=PATCH_LENGTH,
                          n_embd=HIDDEN_SIZE,
                          num_attention_heads=HIDDEN_SIZE // 64,
                          vocab_size=1)
byte_config = GPT2Config(num_hidden_layers=CHAR_NUM_LAYERS,
                         max_length=PATCH_SIZE + 1,
                         max_position_embeddings=PATCH_SIZE + 1,
                         hidden_size=HIDDEN_SIZE,
                         num_attention_heads=HIDDEN_SIZE // 64,
                         vocab_size=128)

model = NotaGenLMHeadModel(encoder_config=patch_config, decoder_config=byte_config).to(device)

def prepare_model_for_kbit_training(model, use_gradient_checkpointing=True):
    """
    Prepare model for k-bit training.
    Features include:
    1. Convert model to mixed precision (FP16).
    2. Disable unnecessary gradient computations.
    3. Enable gradient checkpointing (optional).
    """
    # Convert model to mixed precision
    model = model.to(dtype=torch.float16)

    # Disable gradients for embedding layers
    for param in model.parameters():
        if param.dtype == torch.float32:
            param.requires_grad = False

    # Enable gradient checkpointing
    if use_gradient_checkpointing:
        model.gradient_checkpointing_enable()

    return model


model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=False  
)

print("Parameter Number: " + str(sum(p.numel() for p in model.parameters() if p.requires_grad)))

checkpoint = torch.load(INFERENCE_WEIGHTS_PATH, map_location=torch.device(device))
model.load_state_dict(checkpoint['model'])
model = model.to(device)
model.eval()

def complete_brackets(s):
    stack = []
    bracket_map = {'{': '}', '[': ']', '(': ')'}
    
    # Iterate through each character, handle bracket matching
    for char in s:
        if char in bracket_map:
            stack.append(char)
        elif char in bracket_map.values():
            # Find the corresponding left bracket
            for key, value in bracket_map.items():
                if value == char:
                    if stack and stack[-1] == key:
                        stack.pop()
                    break  # Found matching right bracket, process next character
    
    # Complete missing right brackets (in reverse order of remaining left brackets in stack)
    completion = ''.join(bracket_map[c] for c in reversed(stack))
    return s + completion


def rest_unreduce(abc_lines):

    tunebody_index = None
    for i in range(len(abc_lines)):
        if abc_lines[i].startswith('%%score'):
            abc_lines[i] = complete_brackets(abc_lines[i])
        if '[V:' in abc_lines[i]:
            tunebody_index = i
            break

    metadata_lines = abc_lines[: tunebody_index]
    tunebody_lines = abc_lines[tunebody_index:]

    part_symbol_list = []
    voice_group_list = []
    for line in metadata_lines:
        if line.startswith('%%score'):
            for round_bracket_match in re.findall(r'\((.*?)\)', line):
                voice_group_list.append(round_bracket_match.split())
            existed_voices = [item for sublist in voice_group_list for item in sublist]
        if line.startswith('V:'):
            symbol = line.split()[0]
            part_symbol_list.append(symbol)
            if symbol[2:] not in existed_voices:
                voice_group_list.append([symbol[2:]])
    z_symbol_list = []  # voices that use z as rest
    x_symbol_list = []  # voices that use x as rest
    for voice_group in voice_group_list:
        z_symbol_list.append('V:' + voice_group[0])
        for j in range(1, len(voice_group)):
            x_symbol_list.append('V:' + voice_group[j])

    part_symbol_list.sort(key=lambda x: int(x[2:]))

    unreduced_tunebody_lines = []

    for i, line in enumerate(tunebody_lines):
        unreduced_line = ''

        line = re.sub(r'^\[r:[^\]]*\]', '', line)

        pattern = r'\[V:(\d+)\](.*?)(?=\[V:|$)'
        matches = re.findall(pattern, line)

        line_bar_dict = {}
        for match in matches:
            key = f'V:{match[0]}'
            value = match[1]
            line_bar_dict[key] = value

        # calculate duration and collect barline
        dur_dict = {}  
        for symbol, bartext in line_bar_dict.items():
            right_barline = ''.join(re.split(Barline_regexPattern, bartext)[-2:])
            bartext = bartext[:-len(right_barline)]
            try:
                bar_dur = calculate_bartext_duration(bartext)
            except:
                bar_dur = None
            if bar_dur is not None:
                if bar_dur not in dur_dict.keys():
                    dur_dict[bar_dur] = 1
                else:
                    dur_dict[bar_dur] += 1

        try:
            ref_dur = max(dur_dict, key=dur_dict.get)
        except:
            pass    # use last ref_dur

        if i == 0:
            prefix_left_barline = line.split('[V:')[0]
        else:
            prefix_left_barline = ''

        for symbol in part_symbol_list:
            if symbol in line_bar_dict.keys():
                symbol_bartext = line_bar_dict[symbol]
            else:
                if symbol in z_symbol_list:
                    symbol_bartext = prefix_left_barline + 'z' + str(ref_dur) + right_barline
                elif symbol in x_symbol_list:
                    symbol_bartext = prefix_left_barline + 'x' + str(ref_dur) + right_barline
            unreduced_line += '[' + symbol + ']' + symbol_bartext

        unreduced_tunebody_lines.append(unreduced_line + '\n')

    unreduced_lines = metadata_lines + unreduced_tunebody_lines

    return unreduced_lines


def inference_patch(period, composer, instrumentation, additional_prompts=[], tune_body = []):

    prompt_lines=[
    '%' + period + '\n',
    '%' + composer + '\n',
    '%' + instrumentation + '\n']

    for prompt in additional_prompts:
        prompt_lines.append(prompt)

    for prompt in tune_body:
        prompt_lines.append(prompt)

    tunebody_flag = tune_body != []

    while True:

        failure_flag = False

        bos_patch = [patchilizer.bos_token_id] * (PATCH_SIZE - 1) + [patchilizer.eos_token_id]

        start_time = time.time()

        prompt_patches = patchilizer.patchilize_metadata(prompt_lines)
        byte_list = list(''.join(prompt_lines))
        context_tunebody_byte_list = []
        metadata_byte_list = []

        print(''.join(byte_list), end='')

        prompt_patches = [[ord(c) for c in patch] + [patchilizer.special_token_id] * (PATCH_SIZE - len(patch)) for patch
                          in prompt_patches]
        prompt_patches.insert(0, bos_patch)

        input_patches = torch.tensor(prompt_patches, device=device).reshape(1, -1)

        end_flag = False
        cut_index = None

        tunebody_flag = tunebody_flag == True

        with torch.inference_mode():
            
            while True:
                with torch.autocast(device_type='cuda', dtype=torch.float16):
                    predicted_patch = model.generate(input_patches.unsqueeze(0),
                                                    top_k=TOP_K,
                                                    top_p=TOP_P,
                                                    temperature=TEMPERATURE)
                if not tunebody_flag and patchilizer.decode([predicted_patch]).startswith('[r:'):  # 初次进入tunebody，必须以[r:0/开头
                    tunebody_flag = True
                    r0_patch = torch.tensor([ord(c) for c in '[r:0/']).unsqueeze(0).to(device)
                    temp_input_patches = torch.concat([input_patches, r0_patch], axis=-1)
                    predicted_patch = model.generate(temp_input_patches.unsqueeze(0),
                                                    top_k=TOP_K,
                                                    top_p=TOP_P,
                                                    temperature=TEMPERATURE)
                    predicted_patch = [ord(c) for c in '[r:0/'] + predicted_patch
                if predicted_patch[0] == patchilizer.bos_token_id and predicted_patch[1] == patchilizer.eos_token_id:
                    end_flag = True
                    break
                next_patch = patchilizer.decode([predicted_patch])

                for char in next_patch:
                    byte_list.append(char)
                    if tunebody_flag:
                        context_tunebody_byte_list.append(char)
                    else:
                        metadata_byte_list.append(char)
                    print(char, end='')

                patch_end_flag = False
                for j in range(len(predicted_patch)):
                    if patch_end_flag:
                        predicted_patch[j] = patchilizer.special_token_id
                    if predicted_patch[j] == patchilizer.eos_token_id:
                        patch_end_flag = True

                predicted_patch = torch.tensor([predicted_patch], device=device)  # (1, 16)
                input_patches = torch.cat([input_patches, predicted_patch], dim=1)  # (1, 16 * patch_len)

                if len(byte_list) > 102400:
                    failure_flag = True
                    break
                if time.time() - start_time > 10 * 60: 
                    failure_flag = True
                    break

                if input_patches.shape[1] >= PATCH_LENGTH * PATCH_SIZE and not end_flag:
                    print('Stream generating...')

                    metadata = ''.join(metadata_byte_list)
                    context_tunebody = ''.join(context_tunebody_byte_list)

                    if '\n' not in context_tunebody:
                        break   # Generated content is all metadata, abandon

                    context_tunebody_liness = context_tunebody.split('\n')
                    if not context_tunebody.endswith('\n'):
                        context_tunebody_liness = [context_tunebody_liness[i] + '\n' for i in range(len(context_tunebody_liness) - 1)] + [context_tunebody_liness[-1]]
                    else:
                        context_tunebody_liness = [context_tunebody_liness[i] + '\n' for i in range(len(context_tunebody_liness))]

                    cut_index = len(context_tunebody_liness) // 2
                    abc_code_slice = metadata + ''.join(context_tunebody_liness[-cut_index:])

                    input_patches = patchilizer.encode_generate(abc_code_slice)

                    input_patches = [item for sublist in input_patches for item in sublist]
                    input_patches = torch.tensor([input_patches], device=device)
                    input_patches = input_patches.reshape(1, -1)

                    context_tunebody_byte_list = list(''.join(context_tunebody_lines[-cut_index:]))

            if not failure_flag:
                abc_text = ''.join(byte_list)

                # unreduce
                abc_lines = abc_text.split('\n')
                abc_lines = list(filter(None, abc_lines))
                abc_lines = [line + '\n' for line in abc_lines]
                try:
                    unreduced_abc_lines = rest_unreduce(abc_lines)
                except:
                    failure_flag = True
                    pass
                else:
                    unreduced_abc_lines = [line for line in unreduced_abc_lines if not(line.startswith('%') and not line.startswith('%%'))]
                    unreduced_abc_lines = ['X:1\n'] + unreduced_abc_lines
                    unreduced_abc_text = ''.join(unreduced_abc_lines)
                    return unreduced_abc_text

Parameter Number: 515738880


In [6]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("cuda version built:", torch.version.cuda)

torch: 2.3.0
cuda available: True
cuda version built: 11.8


In [ ]:
periods = {
    "Baroque": [
        # "Bach, Johann Sebastian",
        # "Corelli, Arcangelo",
        # "Handel, George Frideric",
        # "Scarlatti, Domenico",
        "Vivaldi, Antonio"
    ],
    "Classical": [
        "Beethoven, Ludwig van",
        "Haydn, Joseph",
        "Mozart, Wolfgang Amadeus",
        "Paradis, Maria Theresia von",
        "Reichardt, Louise",
        "Saint-Georges, Joseph Bologne",
        "Schroter, Corona"
    ],
    "Romantic": [
        "Bartok, Bela",
        "Berlioz, Hector",
        "Bizet, Georges",
        "Boulanger, Lili",
        "Boulton, Harold",
        "Brahms, Johannes",
        "Burgmuller, Friedrich",
        "Butterworth, George",
        "Chaminade, Cecile",
        "Chausson, Ernest",
        "Chopin, Frederic",
        "Cornelius, Peter",
        "Debussy, Claude",
        "Dvorak, Antonin",
        "Faisst, Clara",
        "Faure, Gabriel",
        "Franz, Robert",
        "Gonzaga, Chiquinha",
        "Grandval, Clemence de",
        "Grieg, Edvard",
        "Hensel, Fanny",
        "Holmes, Augusta Mary Anne",
        "Jaell, Marie",
        "Kinkel, Johanna",
        "Kralik, Mathilde",
        "Lang, Josephine",
        "Lehmann, Liza",
        "Liszt, Franz",
        "Mayer, Emilie",
        "Medtner, Nikolay",
        "Mendelssohn, Felix",
        "Munktell, Helena",
        "Parratt, Walter",
        "Prokofiev, Sergey",
        "Rachmaninoff, Sergei",
        "Ravel, Maurice",
        "Saint-Saens, Camille",
        "Satie, Erik",
        "Schubert, Franz",
        "Schumann, Clara",
        "Schumann, Robert",
        "Scriabin, Aleksandr",
        "Shostakovich, Dmitry",
        "Sibelius, Jean",
        "Smetana, Bedrich",
        "Tchaikovsky, Pyotr"
    ]
}


def sanitize(name: str) -> str:
    """Convert to safe filename (remove spaces, commas, etc.)."""
    return re.sub(r'[^A-Za-z0-9]+', '_', name).strip('_')

instrumentations = [
                    # "Keyboard",
                    "Orchestral", 
                    "Choral", 
                    "Chamber", 
                    "Art Song"
                    ]

for period, composers in periods.items():
    for composer in composers:
        for instrumentation in instrumentations:
            print(f"Period: {period}, Composer: {composer}, Instrumentation: {instrumentation}")
            

            try:
                result = inference_patch(period, composer, instrumentation)

                abc_lines = result.splitlines()
                abc_lines = [line + '\n' for line in abc_lines if line.strip()]  # Add newlines and remove empty lines

                abc_lines = rest_unreduce(abc_lines)

                # Construct unique file name
                filename = f"{sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}.abc"
                
                with open(filename, "w", encoding="utf-8") as f:
                    f.writelines(abc_lines)
                !python abc2xml.py -o . {filename}
            except Exception as e:
                print(f"Some error {e} has occured, skipping{filename}")
                continue


Period: Baroque, Composer: Vivaldi, Antonio, Instrumentation: Orchestral
%Baroque
%Vivaldi, Antonio
%Orchestral

%Orchestral
%end
%%score 1 [ 2 3 4 5 ]
L:1/8
Q:1/4=60
M:4/4
K:Bb
V:1 treble nm="Violino \nprincipale"
V:2 treble nm="Violino I\nripieno"
V:3 treble nm="Violino II\nRipieno"
V:4 alto nm="Viola"
V:5 bass nm="Basso"
[r:0/93][V:1]"^Allegro" d|
[r:1/92][V:1]gGBd g/b/a/g/ d'/b/a/g/|[V:2]z4 z2 z d|[V:3]z4 z2 z B|[V:4]z4 z2 z G|[V:5]z4 z2 z G,|
[r:2/91][V:1]^f2 z d a/g/f/=e/ d/c'/b/a/|[V:2]aD^FA d2 z d|[V:3]d2 z ^f a2 z f|[V:4]^F2 z d F2 z A|[V:5]DD,^F,A, D2 z D,|
[r:3/90][V:1]bg z d g/f/e/d/ c/b/a/g/|[V:2]gGBd g2 z g|[V:3]d2 z B d2 z =e|[V:4]B2 z B B2 z c|[V:5]G,G,,B,,D, G,2 z C,|
[r:4/89][V:1]af z c f/e/d/c/ B/f/e/d/|[V:2]c'FAc f2 z f|[V:3]f2 z f a2 z d|[V:4]A2 z A A2 z B|[V:5]F,F,,A,,C,

KeyboardInterrupt: 

In [18]:

def sanitize(name: str) -> str:
    """Convert to safe filename (remove spaces, commas, etc.)."""
    return re.sub(r'[^A-Za-z0-9]+', '_', name).strip('_')

for i in range(10):
    try:
        period, composer, instrumentation = 'Classical', 'Beethoven, Ludwig van', 'Keyboard'
        default_note, tempo, meter, key = "1/8", "1/4=60", "3/4", "C"
        # score = "1 { ( 2 4 ) | ( 3 5 ) }"
        score = "{ ( 1 2 ) | ( 3 4 ) }"
        voices = ["""V:1 treble nm="piano"\n""",
                    """V:2 treble\n""",
                    """V:3 bass nm="piano"\n""",
                    """V:4 bass\n"""]
        first_tune = """[r:0/80][V:1]"^Andante con moto"!p! g4{a} (gf/e/)|[V:3](C/G/E/G/) (C/G/E/G/) (C/G/E/G/)|\n"""
        result = inference_patch(period, composer, instrumentation, [f"%%score {score} \n", f"L:{default_note}\n", f"Q:{tempo}\n", f"M:{meter}\n", f"K:{key}\n"]+ voices, [first_tune])

        abc_lines = result.splitlines()
        abc_lines = [line + '\n' for line in abc_lines if line.strip()]  # Add newlines and remove empty lines

        abc_lines = rest_unreduce(abc_lines)

        # Construct unique file name
        filename = f"fine_tuned_{sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}_{i}.abc"

        with open(filename, "w", encoding="utf-8") as f:
            f.writelines(abc_lines)
        !python abc2xml.py -o . {filename}
    except Exception as e:
        print(f"Exception caught {e}, skipping {sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}")

%Classical
%Beethoven, Ludwig van
%Keyboard
%%score { ( 1 2 ) | ( 3 4 ) } 
L:1/8
Q:1/4=60
M:3/4
K:C
V:1 treble nm="piano"
V:2 treble
V:3 bass nm="piano"
V:4 bass
[r:0/80][V:1]"^Andante con moto"!p! g4{a} (gf/e/)|[V:3](C/G/E/G/) (C/G/E/G/) (C/G/E/G/)|
[r:1/79][V:1]^cd) d2 z2|[V:3](B,/G/D/G/) (B,/G/D/G/) (B,/G/D/G/)|
[r:2/78][V:1](d3 e/f/ g/a/g/f/)|[V:3](B,/G/D/G/) (B,/G/D/G/) (B,/G/D/G/)|
[r:3/77][V:1](^de) e2 z2|[V:3](C/G/E/G/) (C/G/E/G/) (C/G/E/G/)|
[r:4/76][V:1]!f! (c'>a) (c'7/2 a/4f/4)|[V:3][K:treble] (C/A/F/A/) (C/A/F/A/) (C/A/F/A/)|[V:4][K:treble] x6|
[r:5/75][V:1](c'>g) (c'7/2 g/4e/4)|[V:3](C/G/E/G/) (C/G/E/G/) (C/G/E/G/)|
[r:6/74][V:1]!p! (g/f/e/d/) c2{e} (d/c/d/e/)|[V:3](A,/F/D/F/) (G,/E/C/E/) (G,/F/D/F/)|
[r:7/73][V:1]!f! c2 z/ .c/.d/.e/ .f/.g/.a/.b/|[V:3](C/G/E/G/) (C/G/E/G/) (C/G/E/G/)|
[r:8/72][V:1]!p! (c'>a) (c'7/2 a/4f/4)|[V:3](C/A/F/A/) (C/A/F/A/) (C/A/F/A/)|
[r:9/71][V:1](c'>g) (c'7/2 g/4e/4)|[V:3](C/G/E/G/) (C/G/E/G/) (C/G/E/G/)|
[r:10/70][V:1](g/f/e/d/) c2{e} (d/c/d/e

KeyboardInterrupt: 

In [22]:
try:
    period, composer, instrumentation = 'Classical', 'Beethoven, Ludwig van', 'Art Song'
    default_note, tempo, meter, key = "1/8", "1/4=80", "3/4", "D"
    score = "1 { ( 2 4 ) | ( 3 5 ) }"
    # score = "[ 1 2 3 4 ]"
    # score = ""
    # print([f"%%score {score} \n", f"L:{default_note}\n", f"Q:{tempo}\n", f"M:{meter}\n", f"K:{key}\n"])
    result = inference_patch(period, composer, instrumentation, [f"%%score {score} \n", 
    f"L:{default_note}\n", 
    f"Q:{tempo}\n", 
    f"M:{meter}\n", 
    f"K:{key}\n"])

    abc_lines = result.splitlines()
    abc_lines = [line + '\n' for line in abc_lines if line.strip()]  # Add newlines and remove empty lines

    abc_lines = rest_unreduce(abc_lines)

    # Construct unique file name
    filename = f"{sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}_{i}.abc"

    with open(filename, "w", encoding="utf-8") as f:
        f.writelines(abc_lines)
    !python abc2xml.py -o . {filename}
except Exception as e:
    print(f"Exception caught {e}, skipping {sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}")

%Classical
%Beethoven, Ludwig van
%Art Song
%%score 1 { ( 2 4 ) | ( 3 5 ) } 
L:1/8
Q:1/4=80
M:3/4
K:D
V:1 treble nm="Singstimme\nVoice"
V:2 treble nm="Pianoforte"
V:4 treble 
V:3 bass 
V:5 bass 
[r:0/37][V:1]"^Etwas langsam" z6|[V:3]!pp!{^G,,} [D,,A,,]6|
[r:1/36][V:3]{^G,,} [D,,A,,]6|
[r:2/35][V:2](D/!pp!F/A/F/ GEFD)|[V:3][D,,A,,]6|
[r:3/34][V:2](A,/C/E/G/ FDEC)|[V:3][D,,A,,]6|
[r:4/33][V:2](A,/D/F/A/ [EG][DF][CE][DF])|[V:3][D,,A,,]6|
[r:5/32][V:2]([EG][CA]) ([DF]>[A,E] [F,D]2)|[V:3][D,,A,,]6|
[r:6/31][V:2][A,E]6|[V:3][D,,A,,]6|
[r:7/30][V:2][A,F]6|:[V:3][D,,A,,]6|:
[r:8/29][V:1]!pp! (DF) A d A F|[V:2][A,D]6|[V:3][D,,A,,]6|
[r:9/28][V:1](A3/2 G/) E2 F2|[V:2](E4 F2)|[V:3][D,,A,,]6|[V:4]C4 D2|
[r:10/27][V:1](A/G/) (F/G/) E2 z F|[V:2]([DG]2 [CE]4)|[V:3](D,2 E,4)|[V:5]A,,6|
[r:11/26][V:1]!<(! G A!<)!!>(! B e (c/B/) A!>)!|[V:2][A,E]4 [A,E]2|[V:3][D,,A,,]4 [D,,A,,]2|
[r:12/25][V:1](A/G/F/G/) F2 z2|[V:2]([A,CE]2 [A,DF])(.[DF].[EG].[FA])|[V:3][D,,A,,]4 z2|
[r:13/24][V:1]!mf! (DF) A d A F|[V:2]

In [7]:
try:
    period, composer, instrumentation = 'Classical', 'Haydn, Joseph', 'Chamber'
    default_note, tempo, meter, key = "1/8", "1/4=60", "3/4", "C"
    score = "1 { ( 2 4 ) | ( 3 5 ) }"
    # score = "{ ( 1 2 ) | ( 3 4 ) }"
    result = inference_patch(period, composer, instrumentation, [f"%%score {score} \n", f"L:{default_note}\n", f"Q:{tempo}\n", f"M:{meter}\n", f"K:{key}\n"])

    abc_lines = result.splitlines()
    abc_lines = [line + '\n' for line in abc_lines if line.strip()]  # Add newlines and remove empty lines

    abc_lines = rest_unreduce(abc_lines)

    # Construct unique file name
    filename = f"{sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}_{i}.abc"

    with open(filename, "w", encoding="utf-8") as f:
        f.writelines(abc_lines)
    !python abc2xml.py -o . {filename}
except Exception as e:
    print(f"Exception caught {e}, skipping {sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}")

%Classical
%Haydn, Joseph
%Chamber
%%score 1 { ( 2 4 ) | ( 3 5 ) } 
L:1/8
Q:1/4=60
M:3/4
K:C
V:1 treble nm="Violino"
V:2 treble nm=""
V:4 treble
V:3 bass
V:5 bass
[r:0/101][V:1]"^Andante cantabile" (g3/2a/4b/4) (c'/g/e/c/) (a/f/d/c/)|[V:2][EGc]4 [FAc]2|[V:3]C,2 E,2 F,2|
[r:1/100][V:1](Tc3/2B/4c/4) B2 z2|[V:2][EGc]2 [DGB]2 [GBd]2|[V:3]G,2 G,,G,D,B,,|
[r:2/99][V:1](f3/2g/4a/4) (g/f/e/d/) (c/B/A/G/)|[V:2][Gdf]4 [GBd]2|[V:3]G,,2 G,2 F,2|
[r:3/98][V:1](c>d) (Td3/2c/4d/4) e z|[V:2][Gc]2 [DGB]2 [EGc]2|[V:3]E,2 G,2 C2|
[r:4/97][V:1](f3/2g/4a/4) (g/f/e/d/) (c/B/A/G/)|[V:2][FGd]4 [DGB]2|[V:3]B,2 B,,2 F,2|
[r:5/96][V:1](c>d) (Td3/2c/4d/4) e z|[V:2][CGc]2 [FBd]2 [EGc]2|[V:3]E,2 G,2 C2|
[r:6/95][V:1](^g/a/=g/f/) (e/d/^c/d/) (c/d/f/d/)|[V:2][GAe]2 [FAd]4|[V:3]F,2 F,,2 F,2|
[r:7/94][V:1](c2 B2) z2|[V:2][EGc]2 [DGB]2 [GBd]2|[V:3]G,2 G,,G,D,B,,|
[r:8/93][V:1](d3/2e/4^f/4) (g/d/B/G/) (e/c/B/A/)|[V:2][Gdg]4 [Gce]2|[V:3]G,,2 B,,2 C,2|
[r:9/92][V:1](TG3/2^F/4G/4) F2 z2|[V:2][GBd]2 [^FAd]2 [Ad^f]2|[V:3]D,2 

In [9]:
try:
    period, composer, instrumentation = 'Classical', 'Beethoven, Ludwig van', 'Keyboard'
    default_note, tempo, meter, key = "1/8", "1/4=60", "3/4", "C"
    # score = "1 { ( 2 4 ) | ( 3 5 ) }"
    score = "{ ( 1 2 ) | ( 3 4 ) }"
    result = inference_patch(period, composer, instrumentation, [f"%%score {score} \n", f"L:{default_note}\n", f"Q:{tempo}\n", f"M:{meter}\n", f"K:{key}\n"])

    abc_lines = result.splitlines()
    abc_lines = [line + '\n' for line in abc_lines if line.strip()]  # Add newlines and remove empty lines

    abc_lines = rest_unreduce(abc_lines)

    # Construct unique file name
    filename = f"{sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}_{i}.abc"

    with open(filename, "w", encoding="utf-8") as f:
        f.writelines(abc_lines)
    !python abc2xml.py -o . {filename}
except Exception as e:
    print(f"Exception caught {e}, skipping {sanitize(period)}_{sanitize(composer)}_{sanitize(instrumentation)}")

%Classical
%Beethoven, Ludwig van
%Keyboard
%%score { ( 1 2 ) | ( 3 4 ) } 
L:1/8
Q:1/4=60
M:3/4
K:C
V:1 treble nm="piano"
V:2 treble 
V:3 bass nm="piano"
V:4 bass 
[r:0/135][V:1]"^Andante cantabile"!p! (c3 d/c/ B/c/e/c/)|[V:3]EGEGEG|[V:4]C2 C2 C2|
[r:1/134][V:1](^A2 B2) z2|[V:3]FGFGFG|[V:4]D2 D2 D2|
[r:2/133][V:1](g3 a/g/ ^f/g/a/=f/)|[V:3]DGDGDG|[V:4]B,2 B,2 B,2|
[r:3/132][V:1](^d2 e2) z2|[V:3]EGEGEG|[V:4]C2 C2 C2|
[r:4/131][V:1]!f! (g3/2a/4g/4 c').b.a.g|[V:3]EGEGEG|[V:4]C2 C2 C2|
[r:5/130][V:1](g/^f/c'/b/) (b/a/)(a/g/) (g/f/)(f/e/)|[V:3]DGDGDG|[V:4]B,2 B,2 B,2|
[r:6/129][V:1](e/d/^g/a/) =g3 b/a/|[V:3]EADBC^F|[V:4]C2 D2 D,2|
[r:7/128][V:1]g2 z2 z2::[V:3]B,/G/^F/G/ D/G/B,/D/ G,2::
[r:8/127][V:1]!p! (d3 e/d/ g/f/e/d/)|[V:3][K:treble] GBGBGB|[V:4][K:treble] F2 F2 F2|
[r:9/126][V:1](d>^d e2) z2|[V:3]GcGcGc|[V:4]E2 E2 E2|
[r:10/125][V:1](g3 a/g/ c'/_b/a/g/)|[V:3]GcGcGc|[V:4]E2 E2 E2|
[r:11/124][V:1](g>^g a2) z2|[V:3]AcAcAc|[V:4]F2 F2 F2|
[r:12/123][V:1]!f! (a3 b/c'/ d'/c'/b/a/)|[V:3]DADADA|